In [10]:
%pip install -q youtube-transcript-api langchain-community langchain-google-genai faiss-cpu tiktoken python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install langchain

  Using cached langchain-1.0.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached langgraph-1.0.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached langgraph_checkpoint-3.0.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached langgraph_prebuilt-1.0.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached langgraph_sdk-0.2.9-py3-none-any.whl.metadata (1.5 kB)
  Using cached xxhash-3.6.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached ormsgpack-1.11.0-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
Using cached langchain-1.0.2-py3-none-any.whl (107 kB)
Using cached langgraph-1.0.1-py3-none-any.whl (155 kB)
Using cached langgraph_checkpoint-3.0.0-py3-none-any.whl (46 kB)
Using cached langgraph_prebuilt-1.0.1-py3-none-any.whl (28 kB)
Using cached langgraph_sdk-0.2.9-py3-none-any.whl (56 kB)
Using cached xxhash-3.6.0-cp312-cp312-win_amd64.whl (31 kB)
Using cached ormsgpack-1.11.0-cp312-cp312-win_amd64.whl (112 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_community.document_loaders import YoutubeLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

In [7]:
video_url = "https://www.youtube.com/watch?v=Gfr50f6ZBvo"

try:
    # Initialize and load transcript
    loader = YoutubeLoader.from_youtube_url(video_url, add_video_info=False)
    docs = loader.load()

    # Combine all chunks into a single string
    transcript = " ".join(doc.page_content for doc in docs)

except Exception as e:
    print("Transcript could not be loaded:", e)

In [14]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1700, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [15]:
len(chunks)

90

In [16]:
load_dotenv()
key = os.getenv("GOOGLE_API_KEY")

In [22]:
import time 
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=key)

for i in range(0,len(chunks),10):
    if(i==0):
        vectorstore = FAISS.from_documents(chunks[i:i+10], embedding=embeddings)
    else:
        vectorstore.add_documents(chunks[i:i+10])
    print(f"Added documents {i} to {i+10}")
    time.sleep(10)

Added documents 0 to 10
Added documents 10 to 20
Added documents 20 to 30
Added documents 30 to 40
Added documents 40 to 50
Added documents 50 to 60
Added documents 60 to 70
Added documents 70 to 80
Added documents 80 to 90
